# file_system

In [ ]:
open sm'_operators
open rust_operators

()



In [ ]:
// // test

open testing

()



## types

In [ ]:
inl types () =
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::io::Error\")>] type std_io_Error = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::path::Path\")>] type std_path_Path = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::path::PathBuf\")>] type std_path_PathBuf = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"async_walkdir::DirEntry\")>] type async_walkdir_DirEntry = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"async_walkdir::WalkDir\")>] type async_walkdir_WalkDir = class end"

()



In [ ]:
inl types () =
    global "#if FABLE_COMPILER // file_system.types"
    sm'.types ()
    rust.types ()
    types ()
    global "#endif // file_system.types"

()



## file_mode

In [ ]:
nominal file_mode' = $'System.IO.FileMode'

union file_mode =
    | ModeCreateNew
    | ModeCreate
    | ModeOpen
    | ModeOpenOrCreate
    | Truncate
    | Append

inl file_mode = function
    | ModeCreateNew => $'System.IO.FileMode.CreateNew' : file_mode'
    | ModeCreate => $'System.IO.FileMode.Create' : file_mode'
    | ModeOpen => $'System.IO.FileMode.Open' : file_mode'
    | ModeOpenOrCreate => $'System.IO.FileMode.OpenOrCreate' : file_mode'
    | Truncate => $'System.IO.FileMode.Truncate' : file_mode'
    | Append => $'System.IO.FileMode.Append' : file_mode'

()



## file_access

In [ ]:
nominal file_access' = $'System.IO.FileAccess'

union file_access =
    | AccessRead
    | AccessWrite
    | AccessReadWrite

inl file_access = function
    | AccessRead => $'System.IO.FileAccess.Read' : file_access'
    | AccessWrite => $'System.IO.FileAccess.ReadWrite' : file_access'
    | AccessReadWrite => $'System.IO.FileAccess.ReadWrite' : file_access'

()



## file_share

In [ ]:
nominal file_share' = $'System.IO.FileShare'

union file_share =
    | ShareNone
    | ShareRead
    | ShareWrite
    | ShareReadWrite
    | ShareDelete

inl file_share = function
    | ShareNone => $'System.IO.FileShare.None' : file_share'
    | ShareRead => $'System.IO.FileShare.Read' : file_share'
    | ShareWrite => $'System.IO.FileShare.Write' : file_share'
    | ShareReadWrite => $'System.IO.FileShare.ReadWrite' : file_share'
    | ShareDelete => $'System.IO.FileShare.Delete' : file_share'

()



## file_stream

In [ ]:
nominal file_stream' = $'System.IO.FileStream'

inl file_stream (path : string) mode access share : file_stream' =
    run_target function
        | Fsharp (Native) => fun () =>
            inl mode = mode |> file_mode
            inl access = access |> file_access
            inl share = share |> file_share
            $'new System.IO.FileStream (!path, !mode, !access, !share)'
        | _ => fun () => null ()

()



## path

In [ ]:
nominal path = $'std_path_Path'
nominal path_buf = $'std_path_PathBuf'

()



## new_path_buf

In [ ]:
inl new_path_buf (path : string) : path_buf =
    open rust_operators
    inl path = path |> sm'.to_std_string
    !\\(path, $'"std::path::PathBuf::from($0)"')

()



## path_buf_from

In [ ]:
inl path_buf_from (path : rust.box path) : path_buf =
    open rust_operators
    !\\(path, $'"std::path::PathBuf::from($0)"')

()



## path_buf_join

In [ ]:
inl path_buf_join (s : string) (path_buf : path_buf) : path_buf =
    open rust_operators
    !\\((path_buf, s |> sm'.to_std_string), $'"$0.join($1)"')

()



## path_display

In [ ]:
inl path_display (path : rust.ref' path) : sm'.std_string =
    open rust_operators
    !\\(path, $'"$0.display().to_string()"')

()



## path_buf_file_name

In [ ]:
inl path_buf_file_name (path : path_buf) : optionm'.option' (rust.ref' sm'.os_str) =
    open rust_operators
    !\($'"!path.file_name()"')

()



## path_buf_display

In [ ]:
inl path_buf_display (path_buf : path_buf) : sm'.std_string =
    open rust_operators
    !\\(path_buf, $'"$0.display().to_string()"')

()



## path_buf_exists

In [ ]:
inl path_buf_exists (path_buf : path_buf) : bool =
    open rust_operators
    !\\(path_buf, $'"$0.exists()"')

()



## path_buf_is_dir

In [ ]:
inl path_buf_is_dir (path_buf : path_buf) : bool =
    open rust_operators
    !\\(path_buf, $'"$0.is_dir()"')

()



## path_buf_is_file

In [ ]:
inl path_buf_is_file (path_buf : path_buf) : bool =
    open rust_operators
    !\\(path_buf, $'"$0.is_file()"')

()



## path_buf_parent

In [ ]:
inl path_buf_parent (path_buf : path_buf) : optionm'.option' path_buf =
    open rust_operators
    !\\(path_buf, $'"$0.parent().map(std::path::PathBuf::from)"')

()



## ts_path_join

In [ ]:
inl ts_path_join (b : string) (a : string) : string =
    open ts_operators
    global "type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string"
    inl path : $'IPathJoin' = ts.import_all "path"

    inl a = join a
    inl b = join b
    !\($'"!path.join(!a, !b)"')

()



## (< />)

In [ ]:
inl (</>) (a : string) (b : string) : string =
    run_target function
        | Rust (Native) => fun () =>
            a |> new_path_buf |> path_buf_join b |> path_buf_display |> sm'.from_std_string
        | Fsharp (Native) => fun () =>
            $'System.IO.Path.Combine (!a, !b)'
        | TypeScript _ => fun () =>
            a |> ts_path_join b
        | _ => fun () => null ()

()



## dir_entry

In [ ]:
nominal dir_entry = $'async_walkdir_DirEntry'

()



## walk_dir

In [ ]:
nominal walk_dir = $'async_walkdir_WalkDir'

()



## io_error

In [ ]:
nominal io_error = $"std_io_Error"

()



## dir_entry_path

In [ ]:
inl dir_entry_path (dir_entry : dir_entry) : path_buf =
    !\\(dir_entry, $'"async_walkdir::DirEntry::path(&$0)"')

()



## get_temp_path

In [ ]:
inl get_temp_path () : string =
    $'System.IO.Path.GetTempPath' ()

()



## get_file_name

In [ ]:
inl get_file_name (path : string) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.Path.GetFileName'
        | _ => fun () => null ()

()



## get_full_path

In [ ]:
inl get_full_path (path : string) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            inl path = join path
            path |> $'System.IO.Path.GetFullPath'
        | Rust (Native) => fun () =>
            open rust_operators
            inl path = !\\(path, $'"std::fs::canonicalize(&*$0)"') : resultm.result' path_buf io_error
            path
            |> resultm.unwrap'
            |> path_buf_display
            |> sm'.from_std_string
        | _ => fun () => null ()

()



In [ ]:
// // test

"." |> get_full_path

C:\home\git\polyglot\lib\spiral

let rec method1 () : string =
    let v0 : string = "."
    v0
and method0 () : string =
    let v0 : string option = None
    let v1 : bool = true in let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : string = "std::fs::canonicalize(&*$0)"
    let v3 : string = "."
    let v4 : Result<std_path_PathBuf, std_io_Error> = Fable.Core.RustInterop.emitRustExpr v3 v2
    let v5 : string = "$0.unwrap()"
    let v6 : std_path_PathBuf = Fable.Core.RustInterop.emitRustExpr v4 v5
    let v7 : string = "$0.display().to_string()"
    let v8 : std_string_String = Fable.Core.RustInterop.emitRustExpr v6 v7
    let v9 : string = "fable_library_rust::String_::fromString($0)"
    let v10 : string = Fable.Core.RustInterop.emitRustExpr v8 v9
    v10
#endif
    #if FABLE_COMPILER_RUST && WASM
    let v11 : string = null |> unbox<string>
    v11
#endif
    #if FABLE_COMPILER_RUST && CONTRACT
    let v12 : string = null |> unbox<string>
    v12
#endif
    #if !FABLE_COMPILER &&

In [ ]:
// // test
// // rust=
// // trace=true

types ()
"." |> get_full_path
|> console.write_line

.rs output:
\\?\C:\home\git\polyglot\lib\spiral




00:00:36 debug #1 i: 6 / line: '|> console.write_line' / lastTopLevelIndex:  / finished: False
00:00:36 debug #2 i: 5 / line: '"." |> get_full_path' / lastTopLevelIndex: Some(6) / finished: False
00:00:36 debug #3 i: 4 / line: 'types ()' / lastTopLevelIndex: Some(5) / finished: False
00:00:36 debug #4 i: 3 / line: '' / lastTopLevelIndex: Some(4) / finished: False
00:00:36 debug #5 i: 2 / line: '// // trace=true' / lastTopLevelIndex: Some(4) / finished: False
00:00:36 debug #6 i: 1 / line: '// // rust=' / lastTopLevelIndex: Some(4) / finished: False
00:00:36 debug #7 i: 0 / line: '// // test' / lastTopLevelIndex: Some(4) / finished: False
00:00:37 debug #1 run_with_timeout_async / timeout: 60
00:00:37 debug #8 buildFile / takeWhileInclusive / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:37 debug #9 buildFile / takeWhileInclusive / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:37 debug #10 sendJson / port: 13805 / json: {"FileOpen":{"spiText":"\n\nopen sm\u0027_operators\nop

## get_current_directory

In [ ]:
inl get_current_directory () : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.Directory.GetCurrentDirectory' ()
        | Rust (Native) => fun () =>
            open rust_operators
            inl current_dir = !\($'"std::env::current_dir()"') : resultm.result' path_buf io_error
            current_dir
            |> resultm.unwrap'
            |> path_buf_display
            |> sm'.from_std_string
        | _ => fun () => null ()

()



In [ ]:
// // test

get_current_directory ()

C:\home\git\polyglot\lib\spiral

let rec method0 () : string =
    let v0 : string option = None
    let v1 : bool = true in let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : string = "std::env::current_dir()"
    let v3 : Result<std_path_PathBuf, std_io_Error> = Fable.Core.RustInterop.emitRustExpr () v2
    let v4 : string = "$0.unwrap()"
    let v5 : std_path_PathBuf = Fable.Core.RustInterop.emitRustExpr v3 v4
    let v6 : string = "$0.display().to_string()"
    let v7 : std_string_String = Fable.Core.RustInterop.emitRustExpr v5 v6
    let v8 : string = "fable_library_rust::String_::fromString($0)"
    let v9 : string = Fable.Core.RustInterop.emitRustExpr v7 v8
    v9
#endif
    #if FABLE_COMPILER_RUST && WASM
    let v10 : string = null |> unbox<string>
    v10
#endif
    #if FABLE_COMPILER_RUST && CONTRACT
    let v11 : string = null |> unbox<string>
    v11
#endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !FABLE_COMPILER_TYPESCRIPT && !WASM && !CONTRACT
    let v12 : (un

## create_temp_directory_name

In [ ]:
inl create_temp_directory_name () =
    run_target function
        | Fsharp (Native) => fun () =>
            get_temp_path ()
            </> ($'$"\!{!get_entry_assembly_name ()}"' : string)
            </> (date_time.now () |> date_time.new_guid_from_date_time |> sm'.obj_to_string)
        | target => fun () => null ()

()



In [ ]:
// // test

create_temp_directory_name ()
|> _assert_contains ($'System.IO.Path.DirectorySeparatorChar' : char)

type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string
type [<Struct>] US0 =
    | US0_0 of f0_0 : char
    | US0_1
let rec closure0 () () : string =
    let v0 : string option = None
    let v1 : bool = true in let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : string = null |> unbox<string>
    v2
#endif
    #if FABLE_COMPILER_RUST && WASM
    let v3 : string = null |> unbox<string>
    v3
#endif
    #if FABLE_COMPILER_RUST && CONTRACT
    let v4 : string = null |> unbox<string>
    v4
#endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !FABLE_COMPILER_TYPESCRIPT && !WASM && !CONTRACT
    let v5 : string = System.Reflection.Assembly.GetEntryAssembly().GetName().Name
    v5
#endif
    #if FABLE_COMPILER_TYPESCRIPT
    let v6 : string = null |> unbox<string>
    v6
#endif
    #if FABLE_COMPILER_PYTHON || FABLE_COMPILER_PHP || FABLE_COMPILER_DART
    let v7 : string = Unchecked.defaultof<string>
    v7
#endif
    |> fun x -

## directory_info

In [ ]:
nominal directory_info = $'System.IO.DirectoryInfo'

inl directory_info (path : string) : directory_info =
    path |> $'`directory_info '

()



## directory_info_exists

In [ ]:
inl directory_info_exists (info : directory_info) : bool =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.Exists'
        | _ => fun () => null ()

()



## directory_info_creation_time

In [ ]:
inl directory_info_creation_time (info : directory_info) : date_time.date_time =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.CreationTime'
        | _ => fun () => null ()

()



## directory_info_name

In [ ]:
inl directory_info_name (info : directory_info) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.Name'
        | _ => fun () => null ()

()



## directory_info_full_name

In [ ]:
inl directory_info_full_name (info : directory_info) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.FullName'
        | _ => fun () => null ()

()



## get_source_directory

In [ ]:
inl get_source_directory () =
    $'__SOURCE_DIRECTORY__' : string

()



In [ ]:
// // test

get_source_directory ()
|> directory_info
|> directory_info_name
|> _assert_eq "spiral"

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = __SOURCE_DIRECTORY__
    let v1 : (string -> System.IO.DirectoryInfo) = System.IO.DirectoryInfo 
    let v2 : System.IO.DirectoryInfo = v1 v0
    let v3 : string option = None
    let v4 : bool = true in let mutable _v3 = v3
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v5 : string = null |> unbox<string>
    v5
#endif
    #if FABLE_COMPILER_RUST && WASM
    let v6 : string = null |> unbox<string>
    v6
#endif
    #if FABLE_COMPILER_RUST && CONTRACT
    let v7 : string = null |> unbox<string>
    v7
#endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !FABLE_COMPILER_TYPESCRIPT && !WASM && !CONTRACT
    let v8 : string = v2.Name
    v8
#endif
    #if FABLE_COMPILER_TYPESCRIPT
    let v9 : string = null |> unbox<string>
    v9
#endif
    #if FABLE_COMPILER_PYTHON || FABLE_COMPILER_PHP || FABLE_COMPILER_DART
    let v10 : string = Unchecked.defaultof<string>
    v10
#endif
    |> f

## create_directory

In [ ]:
inl create_directory (path : string) : directory_info =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.Directory.CreateDirectory'
        | _ => fun () => null ()

()



## directory_exists

In [ ]:
inl directory_exists (path : string) : bool =
    run_target function
        | Rust (Native) => fun () =>
            inl path = path |> new_path_buf
            (path |> path_buf_exists) && (path |> path_buf_is_dir)
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.Directory.Exists'
        | TypeScript (Native) => fun () =>
            global "type IFsExistsSync = abstract existsSync: path: string -> bool"
            inl fs : $'IFsExistsSync' = ts.import_all "fs"
            ts.emit_expr path $'"!fs.existsSync($0)"'
        | _ => fun () => null ()

()



## directory_get_parent

In [ ]:
inl directory_get_parent (path : string) : optionm'.option' string =
    run_target function
        | Rust (Native) => fun () =>
            path
            |> new_path_buf
            |> path_buf_parent
            |> optionm'.try'
            |> path_buf_display
            |> sm'.from_std_string
            |> Some
        | Fsharp (Native) => fun () =>
            inl parent : directory_info = path |> $'System.IO.Directory.GetParent'
            if parent =. null ()
            then None
            else parent |> directory_info_full_name |> Some
        | TypeScript _ => fun () =>
            global "type IPathDirname = abstract dirname: path: string -> string"
            inl fs : $'IPathDirname' = ts.import_all "path"
            ts.emit_expr path $'"!fs.dirname($0)"' |> Some
        | _ => fun () => null ()
    |> optionm'.box

()



## file_delete

In [ ]:
inl file_delete (path : string) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.File.Delete'
        | _ => fun () => null ()

()



## file_move

In [ ]:
inl file_move (old_path : string) (new_path : string) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.File.Move (!old_path, !new_path)'
        | _ => fun () => null ()

()



## file_exists

In [ ]:
inl file_exists (path : string) : bool =
    run_target function
        | Rust (Native) => fun () =>
            inl path_buf = path |> new_path_buf
            (path_buf |> path_buf_exists) && (path_buf |> path_buf_is_file)
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.File.Exists'
        | TypeScript (Native) => fun () =>
            global "type IFsExistsSync = abstract existsSync: path: string -> bool"
            inl fs : $'IFsExistsSync' = ts.import_all "fs"
            ts.emit_expr path $'"!fs.existsSync($0)"'
        | _ => fun () => null ()

()



## directory_delete

In [ ]:
inl directory_delete recursive (path : string) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.Directory.Delete (!path, !recursive)'
        | _ => fun () => null ()

()



## read_all_text_async

In [ ]:
inl read_all_text_async (path : string) : _ string =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.File.ReadAllTextAsync' |> async.await_task
        | _ => fun () => null ()

()



## file_exists_content

In [ ]:
inl file_exists_content path content : _ bool =
    run_target function
        | Fsharp (Native) => fun () =>
            fun () =>
                if path |> file_exists |> not
                then false |> return |> fun x => $'(*' : ()
                else
                    $'*) else' : ()
                    inl existing_content = path |> read_all_text_async |> async.let'
                    content = existing_content |> return |> fun x => $'(*' : ()
                |> fun x => $'*)' : ()
            |> async.new_async_unit
        | _ => fun () => null ()

()



## write_all_text

In [ ]:
inl write_all_text (path : string) (text : string) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.File.WriteAllText (!path, !text)'
        | _ => fun () => null ()

()



## write_all_text_async

In [ ]:
inl write_all_text_async (path : string) (text : string) : _ () =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.File.WriteAllTextAsync (!path, !text)' |> async.await_task
        | _ => fun () => null ()

()



## write_all_text_exists

In [ ]:
inl write_all_text_exists path contents =
    fun () =>
        inl exists' = contents |> file_exists_content path |> async.let'
        if not exists'
        then contents |> write_all_text_async path |> async.do
    |> async.new_async

()



## delete_directory_async

In [ ]:
inl delete_directory_async path : _ i64 =
    run_target function
        | Fsharp (Native) => fun () =>
            let rec loop (retry : i64) =
                fun () =>
                    try_unit
                        fun () =>
                            path |> directory_delete true
                            retry |> return
                        fun ex =>
                            if retry % 100i64 = 0 then
                                inl ex = ex |> sm'.format_exception
                                let get_locals () = $'$"path: {!path |> !get_file_name} / ex: {!ex} / {!get_locals ()}"' : string
                                // trace Debug (fun () -> "delete_directory_async") getLocals
                                console.write_line ($'$"delete_directory_async / {!get_locals ()}"' : string)
                            async.sleep 10i32 |> async.do
                            loop (retry + 1) |> async.return_await
                |> async.new_async
            loop 0
        | _ => fun () => null ()

()



## create_temp_directory

In [ ]:
inl create_temp_directory () =
    run_target function
        | Fsharp (Native) => fun () =>
            inl temp_folder = create_temp_directory_name ()
            inl result = temp_folder |> create_directory

            inl exists' = result |> directory_info_exists
            if not exists' then
                inl creation_time = result |> directory_info_creation_time
                inl result = ($'{| Exists = !exists'; CreationTime = !creation_time |}' : any) |> sm'.format_debug
                inl get_locals () : string =
                    $'$"temp_folder: {!temp_folder} / result: {!result} {!get_locals ()}"'
                // trace Debug (fun () => "createTempDirectory") get_locals
                console.write_line ($'$"create_temp_directory / {!get_locals ()}"' : string)
            inl disposable : _ () = new_disposable fun () =>
                temp_folder
                |> delete_directory_async
                |> async.ignore
                |> async.run_synchronously
            temp_folder, disposable
        | _ => fun () => null ()

()



In [ ]:
// // test

inl path, disposable = create_temp_directory ()
disposable |> use |> ignore
path
|> directory_exists
|> _assert_eq true

type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string
type IFsExistsSync = abstract existsSync: path: string -> bool
let rec closure0 () () : string =
    let v0 : string option = None
    let v1 : bool = true in let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : string = null |> unbox<string>
    v2
#endif
    #if FABLE_COMPILER_RUST && WASM
    let v3 : string = null |> unbox<string>
    v3
#endif
    #if FABLE_COMPILER_RUST && CONTRACT
    let v4 : string = null |> unbox<string>
    v4
#endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !FABLE_COMPILER_TYPESCRIPT && !WASM && !CONTRACT
    let v5 : string = System.Reflection.Assembly.GetEntryAssembly().GetName().Name
    v5
#endif
    #if FABLE_COMPILER_TYPESCRIPT
    let v6 : string = null |> unbox<string>
    v6
#endif
    #if FABLE_COMPILER_PYTHON || FABLE_COMPILER_PHP || FABLE_COMPILER_DART
    let v7 : string = Unchecked.defaultof<string>
    v7
#endif
    |> fun x

In [ ]:
// // test

inl lock_directory path =
    fun () =>
        trace Debug (fun () => "_1") get_locals
        "0" |> write_all_text_async (path </> "test.txt") |> async.do
        file_stream
            (path </> "test.txt")
            ModeOpen
            AccessReadWrite
            ShareNone
        |> use
        |> ignore
        trace Debug (fun () => "_2") get_locals
        async.sleep 2000 |> async.do
        trace Debug (fun () => "_3") get_locals
        () |> return
    |> async.new_async

inl temp_dir, disposable = create_temp_directory ()
disposable |> use |> ignore
inl path = temp_dir </> "test"

fun () =>
    trace Debug (fun () => "1") get_locals
    path |> create_directory |> ignore
    trace Debug (fun () => "2") get_locals
    inl child = path |> lock_directory |> async.start_child |> async.let'
    trace Debug (fun () => "3") get_locals
    async.sleep 60 |> async.do
    trace Debug (fun () => "4") get_locals
    inl retries = path |> delete_directory_async |> async.let'
    trace Debug (fun () => "5") get_locals
    child |> async.do
    trace Debug (fun () => "6") get_locals
    retries |> return
|> async.new_async_unit
|> async.run_with_timeout 3000
|> fun x => x : _ i64
|> function
    | Some (retries : i64) =>
        retries
        |> _assert_between
            (if runtime.is_windows () then 50 else 0)
            (if runtime.is_windows () then 150 else 0)

        true
    | _ => false
|> _assert_eq true

type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string
module State = let mutable trace_state = None
type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
    | US0_3
    | US0_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US0}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int64
    | US1_1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int64
    | US2_1
and [<Struct>] US3 =
    | US3_0 of f0_0 : int64
    | US3_1 of f1_0 : exn
and [<Struct>] US4 =
    | US4_0 of f0_0 : int64
    | US4_1 of f1_0 : exn
let rec closure0 () () : string =
    let v0 : string option = None
    let v1 : bool = true in let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : string = null |> unbox<string>
    v2
#endif
    #if FABLE_COMPILER_RUST && WASM
    let v3 : string = null |> unbox<string>
    v3
#endif
    #if FABLE_COMPILER_RUST && CONTRACT
    let v4 : string = null |> unbox<string>
    v4
#endif
    #if !F

## wait_for_file_access

In [ ]:
inl wait_for_file_access access path =
    run_target function
        | Fsharp (Native) => fun () =>
            inl file_access, file_share =
                access
                |> optionm'.default_value (AccessReadWrite, ShareRead)
            let rec loop (retry : i64) : _ i64 =
                fun () =>
                    try_unit
                        fun () =>
                            file_stream
                                path
                                ModeOpen
                                file_access
                                file_share
                            |> use
                            |> ignore
                            retry |> return
                        fun ex =>
                            if retry > 0 && retry % 100i64 = 0 then
                                inl ex = ex |> sm'.format_exception
                                let get_locals () = $'$"path: {!path |> !get_file_name} / retry: {!retry} / ex: {!ex} / {!get_locals ()}"' : string
                                // trace Debug (fun () -> "wait_for_file_access") getLocals
                                console.write_line ($'$"wait_for_file_access / {!get_locals ()}"' : string)
                            async.sleep 10i32 |> async.do
                            loop (retry + 1) |> async.return_await
                |> async.new_async
            loop 0
        | _ => fun () => null ()

inl wait_for_file_access_read path =
    path
    |> wait_for_file_access (Some (
        AccessRead,
        ShareRead
    ))

()



In [ ]:
// // test

inl lock_file path =
    fun () =>
        trace Debug (fun () => "_1") get_locals
        inl stream : file_stream' =
            file_stream
                path
                ModeOpen
                AccessReadWrite
                ShareNone
            |> use
        trace Debug (fun () => "_2") get_locals
        async.sleep 2000 |> async.do
        trace Debug (fun () => "_3") get_locals
        ($'!stream.Seek (0L, System.IO.SeekOrigin.Begin)' : i64) |> ignore
        trace Debug (fun () => "_4") get_locals
        $'!stream.WriteByte' 49u8
        trace Debug (fun () => "_5") get_locals
        stream |> $'_.Flush()'
        trace Debug (fun () => "_6") get_locals
    |> async.new_async

inl temp_dir, disposable = create_temp_directory ()
disposable |> use |> ignore
inl path = temp_dir </> "test.txt"

fun () =>
    trace Debug (fun () => "1") get_locals
    "0" |> write_all_text_async path |> async.do
    trace Debug (fun () => "2") get_locals
    inl child = path |> lock_file |> async.start_child |> async.let'
    trace Debug (fun () => "3") get_locals
    async.sleep 1 |> async.do
    trace Debug (fun () => "4") get_locals
    inl retries = path |> wait_for_file_access None |> async.let'
    trace Debug (fun () => "5") get_locals
    inl text = path |> read_all_text_async |> async.let'
    trace Debug (fun () => "6") get_locals
    child |> async.do
    trace Debug (fun () => "7") get_locals
    (retries, text) |> return
|> async.new_async_unit
|> async.run_with_timeout 3000
|> function
    | Some ((retries : i64), text) =>
        retries
        |> _assert_between
            (if runtime.is_windows () then 50 else 100)
            (if runtime.is_windows () then 150 else 200)

        text |> _assert_eq (join "1")

        true
    | _ => false
|> _assert_eq true

type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string
module State = let mutable trace_state = None
type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
    | US0_3
    | US0_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US0}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int64
    | US1_1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int64
    | US2_1
and [<Struct>] US3 =
    | US3_0 of f0_0 : int64 * f0_1 : string
    | US3_1
and [<Struct>] US4 =
    | US4_0 of f0_0 : int64 * f0_1 : string
    | US4_1 of f1_0 : exn
and [<Struct>] US5 =
    | US5_0 of f0_0 : int64 * f0_1 : string
    | US5_1 of f1_0 : exn
let rec closure0 () () : string =
    let v0 : string option = None
    let v1 : bool = true in let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : string = null |> unbox<string>
    v2
#endif
    #if FABLE_COMPILER_RUST && WASM
    let v3 : string = null |> unbox<string>
    v3
#endif

## read_all_text_retry_async

In [ ]:
inl read_all_text_retry_async full_path : async.async (optionm'.option' string) =
    run_target function
        | Fsharp (Native) => fun () =>
            let rec loop (retry : i64) =
                fun () =>
                    try_unit
                        fun () =>
                            if retry > 0
                            then
                                full_path
                                |> wait_for_file_access_read
                                |> async.run_with_timeout_async 1000
                                |> async.ignore
                                |> async.do
                            full_path |> read_all_text_async |> async.map (Some >> optionm'.box) |> async.return_await
                        fun ex =>
                            if retry <> 0 then
                                inl ex = ex |> sm'.format_exception
                                let get_locals () = $'$"retry: {!retry} / ex: {!ex} / {!get_locals ()}"' : string
                                trace Debug (fun () => $'"read_all_text_retry_async"' : string) get_locals
                                (None : _ string) |> optionm'.box |> return |> fun x => $'(*' : ()
                            else
                                $'*) else' : ()
                                loop (retry + 1) |> async.return_await |> fun x => $'(*' : ()
                            |> fun x => $'*)' : ()
                |> async.new_async
            loop 0
        | _ => fun () => null ()

()



## delete_file_async

In [ ]:
inl delete_file_async path : _ i64 =
    run_target function
        | Fsharp (Native) => fun () =>
            let rec loop (retry : i64) =
                fun () =>
                    try_unit
                        fun () =>
                            path |> file_delete
                            return retry
                        fun ex =>
                            if retry % 100 = 0 then
                                inl ex = ex |> sm'.format_exception
                                let getLocals () = $'$"path: {!path |> !get_file_name} / ex: {ex} / {!get_locals ()}"'
                                trace Warning (fun () => "delete_file_async") get_locals
                            async.sleep 10 |> async.do
                            loop (retry + 1) |> async.return_await
                |> async.new_async
            loop 0
        | _ => fun () => null ()

()



In [ ]:
// // test

inl lock_file path =
    fun () =>
        trace Debug (fun () => "_1") get_locals
        file_stream
            path
            ModeOpen
            AccessReadWrite
            ShareNone
        |> use
        |> ignore
        trace Debug (fun () => "_2") get_locals
        async.sleep 2000 |> async.do
        trace Debug (fun () => "_3") get_locals
    |> async.new_async

inl temp_dir, disposable = create_temp_directory ()
disposable |> use |> ignore
inl path = temp_dir </> "test.txt"

fun () =>
    trace Debug (fun () => "1") get_locals
    "0" |> write_all_text_async path |> async.do
    trace Debug (fun () => "2") get_locals
    inl child = lock_file path |> async.start_child |> async.let'
    trace Debug (fun () => "3") get_locals
    async.sleep 1 |> async.do
    trace Debug (fun () => "4") get_locals
    inl retries = delete_file_async path |> async.let'
    trace Debug (fun () => "5") get_locals
    child |> async.do
    trace Debug (fun () => "6") get_locals
    return retries
|> async.new_async_unit
|> async.run_with_timeout 3000
|> function
    | Some (retries : i64) =>
        retries
        |> _assert_between
            (if runtime.is_windows () then 50 else 0)
            (if runtime.is_windows () then 150 else 0)

        true
    | _ => false
|> _assert_eq true

type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string
module State = let mutable trace_state = None
type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
    | US0_3
    | US0_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US0}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int64
    | US1_1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int64
    | US2_1
and [<Struct>] US3 =
    | US3_0 of f0_0 : int64
    | US3_1 of f1_0 : exn
and [<Struct>] US4 =
    | US4_0 of f0_0 : int64
    | US4_1 of f1_0 : exn
let rec closure0 () () : string =
    let v0 : string option = None
    let v1 : bool = true in let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : string = null |> unbox<string>
    v2
#endif
    #if FABLE_COMPILER_RUST && WASM
    let v3 : string = null |> unbox<string>
    v3
#endif
    #if FABLE_COMPILER_RUST && CONTRACT
    let v4 : string = null |> unbox<string>
    v4
#endif
    #if !F

## move_file_async

In [ ]:
inl move_file_async new_path old_path : _ i64 =
    run_target function
        | Fsharp (Native) => fun () =>
            let rec loop (retry : i64) =
                fun () =>
                    try_unit
                        fun () =>
                            file_move old_path new_path
                            return retry
                        fun ex =>
                            if retry % 100 = 0 then
                                let get_locals () =
                                    inl ex = ex |> sm'.format_exception
                                    $'$"old_path: {!old_path |> !get_file_name} / new_path: {!new_path |> !get_file_name} / ex: {!ex} / {!get_locals ()}"' : string
                                trace Warning (fun () => "move_file_async") get_locals
                            async.sleep 10 |> async.do
                            loop (retry + 1) |> async.return_await
                |> async.new_async_unit
            loop 0
        | _ => fun () => null ()

()



In [ ]:
// // test

inl lock_file path =
    fun () =>
        trace Debug (fun () => "_1") get_locals
        file_stream
            path
            ModeOpen
            AccessReadWrite
            ShareNone
        |> use
        |> ignore
        trace Debug (fun () => "_2") get_locals
        async.sleep 2000 |> async.do
        trace Debug (fun () => "_3") get_locals
    |> async.new_async

inl temp_dir, disposable = create_temp_directory ()
disposable |> use |> ignore
let path = temp_dir </> "test.txt"
let new_path = temp_dir </> "test2.txt"

fun () =>
    trace Debug (fun () => "1") get_locals
    "0" |> write_all_text_async path |> async.do
    trace Debug (fun () => "2") get_locals
    inl child = lock_file path |> async.start_child |> async.let'
    trace Debug (fun () => "3") get_locals
    async.sleep 1 |> async.do
    trace Debug (fun () => "4") get_locals
    inl retries1 = path |> move_file_async new_path |> async.let'
    trace Debug (fun () => "5") get_locals
    inl retries2 = new_path |> wait_for_file_access None |> async.let'
    trace Debug (fun () => "6") get_locals
    inl text = new_path |> read_all_text_async |> async.let'
    trace Debug (fun () => "7") get_locals
    child |> async.do
    trace Debug (fun () => "8") get_locals
    (retries1, retries2, text) |> return
|> async.new_async_unit
|> async.run_with_timeout 3000
|> function
    | Some (retries1, retries2, text) =>
        retries1
        |> _assert_between
            (if runtime.is_windows () then 50i64 else 0)
            (if runtime.is_windows () then 150 else 0)

        retries2
        |> _assert_between
            (if runtime.is_windows () then 0i64 else 100)
            (if runtime.is_windows () then 0 else 200)

        text |> _assert_eq (join "0")

        true
    | _ => false
|> _assert_eq true

type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string
module State = let mutable trace_state = None
type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
    | US0_3
    | US0_4
and Mut0 = {mutable l0 : int64}
and Mut1 = {mutable l0 : bool}
and Mut2 = {mutable l0 : US0}
and [<Struct>] US1 =
    | US1_0 of f0_0 : int64
    | US1_1
and [<Struct>] US2 =
    | US2_0 of f0_0 : int64
    | US2_1
and [<Struct>] US3 =
    | US3_0 of f0_0 : int64 * f0_1 : int64 * f0_2 : string
    | US3_1
and [<Struct>] US4 =
    | US4_0 of f0_0 : int64 * f0_1 : int64 * f0_2 : string
    | US4_1 of f1_0 : exn
and [<Struct>] US5 =
    | US5_0 of f0_0 : int64 * f0_1 : int64 * f0_2 : string
    | US5_1 of f1_0 : exn
let rec closure0 () () : string =
    let v0 : string option = None
    let v1 : bool = true in let mutable _v0 = v0
    #if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : string = null |> unbox<string>
    v2
#endif
    #if FABLE_COMPILER_RUST && WASM
    let v3 :

## find_parent

In [ ]:
inl find_parent name is_file root_dir =
    let rec loop dir =
        if dir </> name |> (if is_file then file_exists else directory_exists)
        then dir
        else
            inl result = dir |> (join directory_get_parent)
            match result |> optionm'.unbox with
            | Some parent => parent |> loop
            | None => failwith $'$"""No parent for {if !is_file then "file" else "dir"} \'{!name}\' at \'{!root_dir}\' (until \'{!dir}\')"""'
    loop root_dir

()



In [ ]:
// // test
// // print_code=false

a ;[ ".paket", false; "paket.dependencies", true ]
|> am.map fun (file, is_file) =>
    get_source_directory ()
    |> find_parent file is_file
    |> directory_info
    |> directory_info_name
|> am'.distinct
|> _assert_eq (a ;[ "polyglot" ] : _ i32 _)

[|"polyglot"|]


In [ ]:
// // test
// // rust=
// // print_code=false

types ()
a ;[ ".paket", false; "paket.dependencies", true ]
|> am.map fun (file, is_file) =>
    fun () =>
        join
            get_source_directory ()
            |> find_parent file is_file
            |> new_path_buf
            |> path_buf_file_name
            |> optionm'.try'
            |> sm'.to_os_string
            |> sm'.os_to_str
            |> optionm'.try'
            |> sm'.ref_to_std_string
            |> sm'.from_std_string
            |> Some
            |> optionm'.box
    |> fun x => x () |> optionm'.unbox
    |> optionm'.default_value ""
|> am'.distinct
|> fun result =>
    result |> am'.length |> _assert_eq 1i32
    index result 0i32 |> _assert_eq "polyglot"

1
"polyglot"


## get_repository_root

In [ ]:
inl get_repository_root () =
    get_source_directory ()
    |> find_parent ("polyglot" </> ".paket") false
    |> fun root => root </> "polyglot"

()



## main

In [ ]:
inl main () =
    types ()
    init_trace_state ()
    $"let delete_directory_async x = !delete_directory_async x" : ()
    $"let wait_for_file_access x = !wait_for_file_access x" : ()
    $"let wait_for_file_access_read x = !wait_for_file_access_read x" : ()
    $"let read_all_text_async x = !read_all_text_async x" : ()
    $"let file_exists_content x = !file_exists_content x" : ()
    $"let write_all_text_async x = !write_all_text_async x" : ()
    $"let write_all_text_exists x = !write_all_text_exists x" : ()
    $"let delete_file_async x = !delete_file_async x" : ()
    $"let move_file_async x = !move_file_async x" : ()
    $"let read_all_text_retry_async x = !read_all_text_retry_async x" : ()
    $"let create_temp_directory_name () = !create_temp_directory_name ()" : ()
    $"let create_temp_directory () = !create_temp_directory ()" : ()
    $"let get_source_directory () = !get_source_directory ()" : ()
    $"let find_parent x = !find_parent x" : ()
    $"let get_repository_root () = !get_repository_root ()" : ()
    inl combine x = (</>) x
    $"let (</>) x = !combine x" : ()

#if FABLE_COMPILER // file_system.types
[<Fable.Core.Erase; Fable.Core.Emit("str")>] type Str = class end
[<Fable.Core.Erase; Fable.Core.Emit("base64::DecodeError")>] type base64_DecodeError = class end
[<Fable.Core.Erase; Fable.Core.Emit("borsh::io::Error")>] type borsh_io_Error = class end
[<Fable.Core.Erase; Fable.Core.Emit("js_sys::JsString")>] type js_sys_JsString = class end
[<Fable.Core.Erase; Fable.Core.Emit("serde_json::Error")>] type serde_json_Error = class end
[<Fable.Core.Erase; Fable.Core.Emit("serde_json::Value")>] type serde_json_Value = class end
[<Fable.Core.Erase; Fable.Core.Emit("serde_wasm_bindgen::Error")>] type serde_wasm_bindgen_Error = class end
[<Fable.Core.Erase; Fable.Core.Emit("std::ffi::OsStr")>] type std_ffi_OsStr = class end
[<Fable.Core.Erase; Fable.Core.Emit("std::ffi::OsString")>] type std_ffi_OsString = class end
[<Fable.Core.Erase; Fable.Core.Emit("std::str::Utf8Error")>] type std_str_Utf8Error = class end
[<Fable.Core.Erase; Fable.Core.Emit("std::s